In [1]:
import pickle

import numpy as np
from tqdm import tqdm
import torch
from scipy.special import expit

from ela import generator

import utils
import metrics
import datasets

# LIST OF EXPERIMENTS TO INCLUDE 
experiments = [
    'nopoolrefinenet_dpn92',
]

test_predictions_experiment = []

for name in experiments:
    test_predictions = utils.TestPredictions('{}'.format(name), mode='val')
    test_predictions_experiment.append(test_predictions.load_raw())

samples_train = utils.get_train_samples()
transforms = generator.TransformationsGenerator([])
dataset = datasets.AnalysisDataset(samples_train, './data/train', transforms, utils.TestPredictions('{}'.format(name), mode='val').load())

train = {}
for id in tqdm(samples_train):
    _, mask, _ = dataset.get_by_id(id)
    test_prediction = np.concatenate([predictions[id] for predictions in test_predictions_experiment], axis=0)
    prediction = np.mean(expit(test_prediction), axis=0)

    train[id] = (prediction, mask)


def strip_nan(e):
    if e != e:
        return None

    return e[:-4]


# PUT CORRECT PATH TO NEIGHBORS FILE HERE
with open('./data/8_neighbours_mosaics.pkl', "rb") as f:
    neighbors = pickle.load(f)
    neighbors = {k[:-4]: [strip_nan(e) for e in v] for k, v in neighbors.items()}

samples_test = utils.get_test_samples()
predictions_test = utils.TestPredictions('{}-split_{}'.format(name, 0)).load()

100%|██████████| 4000/4000 [00:06<00:00, 621.70it/s]                                                


In [2]:
from os.path import join
from skimage.io import imread
from skimage import img_as_float

def get_prediction(id):
    """
    Depdening on the given id either returns training ground-truth mask or test prediction
    """
    if id in samples_train:
        prediction = img_as_float(imread(join('./data/train', 'masks', id) + '.png'))

    if id in samples_test:
        prediction = predictions_test[id]
        
    return prediction

train_postprocessed = {id: (prediction.copy(), mask.copy()) for id, (prediction, mask) in train.items()}
for sample in tqdm(samples_train):
    if sample in neighbors:
        sample_neighbors = neighbors[sample]
        prediction_sample, mask_sample = train_postprocessed[sample]
        
        # HERE IS SOME SAMPLE CODE. PUT YOUR POST_PROCESSING HERE
        n_top, n_left, n_right, n_bottom =  sample_neighbors[1],  sample_neighbors[3],  sample_neighbors[4],  sample_neighbors[6]
        
        if n_top and n_left and n_right and n_bottom:
            top = get_prediction(n_top)
            left = get_prediction(n_left)
            right = get_prediction(n_right)
            bottom = get_prediction(n_bottom)
            
            t = 0.1
            if np.mean(np.abs(top[-1, :] - bottom[0, :]), axis=None) < t:
                prediction_sample[:, :] = top[-1, :]

            train_postprocessed[sample] = prediction_sample, mask_sample
            
        # END SAMPLE CODE


predictions = [prediction for id, (prediction, mask) in train_postprocessed.items()]
predictions = np.stack(predictions, axis=0)
masks = [mask for id, (prediction, mask) in train_postprocessed.items()]
masks = np.stack(masks, axis=0)

predictions = (predictions > 0.5).astype(np.float32)

map = metrics.mAP(torch.FloatTensor(predictions).cuda(), torch.FloatTensor(masks).cuda())

print('mAP', map)


100%|██████████| 4000/4000 [00:00<00:00, 8829.13it/s]


mAP tensor(0.8635, device='cuda:0')
